In [5]:
!rm -r data
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data
# 학습할 데이터 불러오기
!pip install Unidecode
# Unidecode 없으면 install

--2019-11-08 03:48:57--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2019-11-08 03:48:57 (43.1 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import unidecode
import string
import random
import re
import time, math



In [0]:
# 사용될 HyperParameters
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [14]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars 100


In [15]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len = ', file_len)

file_len =  1115394


In [0]:
# 모든 파일을 한번에 학습할수 없기때문에 일정한 크기로 잘라줘야한다
# 랜덤한 위치에서 시작해 일정 크기만큼의 문자열을 읽어오는 random_chunk() 라는 함수를 만들어줌
def random_chunk():
  start_index = random.randint(0,file_len- chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

In [0]:
# 문자열을 입력값과 목표값으로 나누어준다
# random_chunk() 함수를 통해 랜덤한 문자열을 불러와 입력값과 목표값으로 나눠 리턴
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    # 마지막 인덱스만 빼고 char_tensor 에 넣어줌
    target = char_tensor(chunk[1:])
    # 첫번째 인덱스만 빼고 넣어줌
    return inp, target

In [0]:
# 일정 크기로 문자열을 읽어온 다음 이를 앞에서 저장해둔 출력 가능한 문자열 리스트를 통해 인덱스로 바꿔준다
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor('ABCdef'))

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size

        self.encoder = nn.Embedding(input_size,embedding_size)
        # 총 단어의 개수, 임베딩 시킬 벡터의 차원
        # 알파벳이나 단어같은 기본 단위 요소들을 일정한 길이를 가지는 벡터공간에 투영시키는 것
        #self.rnn = nn.RNN(embedding_size,hidden_size,num_layers)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers)
        #self.rnn = nn.GRU(embedding_size, hidden_size , num_layers) # GRU방식
        self.decoder = nn.Linear(hidden_size,output_size)
        # 임베딩 하기전 의 데이터 형태로 돌려놓는 역할


    # def forward(self,input,hidden):
    #   out = self.encoder(input.view(1,-1))
    #   out,hidden = self.rnn(out,hidden)
    #   out = self.decoder(out.view(batch_size,-1))
    #   return out, hidden


    # def init_hidden(self):
    #   hidden = torch.zeros(self.num_layers,batch_size,hidden_size)
    #   return hidden

    #LSTM version
    def forward(self,input,hidden,cell):
      out = self.encoder(input.view(batch_size,-1))
      out,(hidden,cell) = self.rnn(out,(hidden,cell))
      out = self.decoder(out.view(batch_size,-1))
      return out,hidden,cell

    def init_hidden(self):
      hidden = torch.zeros(num_layers,batch_size,hidden_size)
      cell = torch.zeros(num_layers,batch_size,hidden_size)
      return hidden,cell

# 클래스를 인스턴스화 

#LSTM..
model = RNN(n_characters, embedding_size, hidden_size, n_characters,num_layers)

# model = RNN(input_size=n_characters,
#             embedding_size = embedding_size,
#             hidden_size = hidden_size,
#             output_size = n_characters,
#             num_layers = 2)


# A 라는 문자열을 입력으로 주고 char_tensor를 이용해 텐서로 바꿔준다.
# 은식상태도 초기화
inp = char_tensor("A")
hidden = model.init_hidden()
# 임베딩 값과 은식 상태 값을 전달해준다.
# 임베딩 차원이 문자열 크기와 다르기 때문에 RNN 노드의 결과값을 디코더를 통해 다시 맞춰줘야한다. 
out,hidden = model(inp,hidden)

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [0]:
for i in range(num_epochs):
  total = char_tensor(random_chunk())
  inp = total[:-1]
  label = total[:-1]
  hidden = model.init_hidden()
  loss = torch.tensor([0]).type(torch.FloatTensor)

# 더이상 기울기 학습을 하지 못하게 함
  optimizer.zero_grad()
  for j in range(chunk_len-1):
    x = inp[j]
    # unsqueeze()함수는 인수로 받은 위치에 새로운 차원을 삽입한다.
    y_ = label[j].unsqueeze(0).type(torch.LongTensor)
    y,hidden = model(x,hidden)
    loss += loss_func(y,y_)

  loss.backward()
  optimizer.step()

In [0]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)



In [0]:
for i in range(num_epochs):
    inp,label = random_training_set()
    hidden,cell = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")